### Libraries

In [ ]:
import re
import os
import copy
import angr
import pickle
import monkeyhex
import networkx as nx

from time import time
from collections import Counter, deque

### Functions

In [ ]:
def store_as_list_of_dicts(file_name, graph_data):
    """
    Save all graphs into a pickle file which can be later read by other program for intrusion detection
    """
    list_of_dicts = [(func_name, nx.to_dict_of_dicts(graph)) for func_name, graph in graph_data.items()]
    with open(f'{file_name}.pkl', 'wb') as f:
        pickle.dump(list_of_dicts, f)

In [ ]:
def get_node_with_func_name(cfg, func_name = 'main', func_addr = None):
    """
    Retursn FuncNode of a function with name provided along with angr-disassembly data.
    Can use func_addr if found multiple functions with same name
    """
    functions = cfg.functions.values()
    for func in functions:
        if (func_addr is None) and (func.name == func_name):
            return func
        if (func_addr is not None) and (func_name == func.name) and (int(func_addr) == func.addr):
            return func

In [ ]:
def resolve(main_node, sub_node):
    """
    There can be multiple copies of a single function node. This function resolves it 
    by always selecting first one. Need to discuss to fix this.
    """
    result = []
    valid_edges = list(main_node.transition_graph.edges() - main_node.graph.edges())
    for e1, e2 in valid_edges:
        if e2.addr in sub_node:
            result.append(e2)
    if len(result) > 1:
        print(f'Multiple instaces of function  {main_node.name} found, need to fix')
    return result[0].addr

In [ ]:
def get_all_inst_add(disas_string_list):
    """
    Provided with a list of a function disassemble string it gives all the 
    instruction address present for that function
    """
    address_values = []
    for inst_data in disas_string_list:
        inst_split_data = inst_data.strip()
        if inst_split_data != []:
            match = re.findall('(\d\w+)\s+', inst_split_data)
            if match:
                address_values.append(int(match[0], 16))
    return address_values

In [ ]:
def get_missing_sys_call_data(node):
    """
    Incase system call is present but not identified (require Dataflow analysis) angr does some
    dataflow analysis itself and tell which system call it belong to.
    """
    add_sys_call_data = {}
    data = (node.transition_graph.edges() - node.graph.edges())
    for e1, e2 in data:
        if isinstance(e2, angr.knowledge_plugins.functions.function.Function) and e2.is_syscall and (e2.name in valid_sys_calls):
            add_sys_call_data[e1.addr] = e2.name
    return add_sys_call_data

In [ ]:
def is_hex(value):
    """
    Returns boolean value based on whether its a hexadecimal character or not
    """
    try:
        int(value, 16)
        return True
    except Exception as e:
        return False

In [ ]:
def get_graph_label_data(cfg, func_name, valid_func_nodes_names, mapping_data, s_mapping_data):
    """
    Heart of the entire program. This program basically maps all the instruction of a function node
    disassemble string respectively. We consider only "calls" and "jmps" rest all instructions are made
    nops and later resolved to obtain the graph structure. 
    """
    to_resolve = []
    f_name, f_addr = func_name.split('-')
    func_node = get_node_with_func_name(cfg, f_name, f_addr)
    addr_inst_data_string = cfg.project.analyses.Disassembly(func_node).render()
    addr_inst_data = addr_inst_data_string.split('\n')
    current_jumps = 0
    final_data = {}
    ret_addr_values = []
    global counter2
    for label_data in addr_inst_data:
        label_data = label_data.strip()
        matches = re.findall(f'(\d\w+)\s*({jumps_string})\s*(\w+)', label_data)
        matches1 = re.findall(f'(\d\w+)\s*(ret)', label_data)
        if matches:
            if (matches[0][1] == 'call') and (matches[0][2] in valid_func_nodes_names):
                matches = matches[0]
                match_node = mapping_data[matches[2]]
                if len(match_node) > 1:
                    req_addr = resolve(func_node, match_node)
                else:
                    req_addr = match_node[0]
                to_resolve_string = f'{matches[2]}-{req_addr}'
                final_data[int(matches[0], 16)] = to_resolve_string
                to_resolve.append(to_resolve_string)
            elif matches[0][1] != 'call':
                matches = matches[0]
                if not is_hex(matches[2]) and (matches[2] in valid_func_nodes_names):
                    match_node = mapping_data[matches[2]]
                    if len(match_node) > 1:
                        req_addr = resolve(func_node, match_node)
                    else:
                        req_addr = match_node[0]
                    to_resolve_string = f'{matches[2]}-{req_addr}'
                    final_data[int(matches[0], 16)] = to_resolve_string
                    to_resolve.append(to_resolve_string)
                else:
                    final_data[int(matches[0], 16)] = 'ep'
                    counter2 += 1
        if matches1:
            ret_addr_values.append(matches1[0][0])
    addr_values = get_all_inst_add(addr_inst_data)
    start_addr, end_addr = addr_values[0], addr_values[-1]
    remaining_addr = set(addr_values) - set(final_data.keys())
    for index, addr_value in enumerate(remaining_addr):
        final_data[addr_value] = f'nop-{index}'
    add_sys_call_mapping = get_missing_sys_call_data(func_node)
    for addr, sys_call in add_sys_call_mapping.items():
        sys_node = s_mapping_data[sys_call]
        sys_string = f'{sys_call}-{sys_node}'
        final_data[addr] = sys_string
        to_resolve.append(sys_string)
    for ret_addr in ret_addr_values:
        final_data[int(ret_addr, 16)] = f'ret-end'
    return final_data, set(to_resolve)

In [ ]:
def get_degree_nodes(graph_data):
    """
    Returns source and sink among all nodes given graph data
    """
    in_d, out_d = [], []
    self_loop_data = set(nx.nodes_with_selfloops(graph_data))
    for node in graph_data.nodes():
        n_out_d = graph_data.out_degree(node)
        n_in_d = graph_data.in_degree(node)
        if (not n_in_d) or ((n_in_d == 1) and (node in self_loop_data)):
            in_d.append(node)
        if (not n_out_d) or ((n_out_d == 1) and (node in self_loop_data)):
            out_d.append(node)
    return in_d, out_d

In [ ]:
def find_mergable_nodes(graph_data, label_data):
    """
    All the nop which were previosuly marked are now merged together
    """
    final_nodes = []
    for node in graph_data.nodes:
        label_value = label_data[node]
        if not label_value.startswith('nop-'):
            continue
        else:
            final_nodes.append(node)
    return final_nodes

In [ ]:
def merge_nodes(nodes_data, graph_data):
    """
    Arranges the edges and removes node values accordingly.
    """
    for node in nodes_data:
        succ_nodes = list(graph_data.successors(node))
        pre_nodes = list(graph_data.predecessors(node))
        for pre_node in pre_nodes:
            for suc_node in succ_nodes:
                graph_data.add_edge(pre_node, suc_node)
        graph_data.remove_node(node)
    return graph_data

In [ ]:
def to_edge_based(graph_data, label_data, func_name):
    """
    Adds edge labels based on the e2 node and returns the data which will be used in future.
    """
    attrib_data = {}
    for node in graph_data.nodes():
        node_label = label_data[node]
        pre_nodes = list(graph_data.predecessors(node))
        for pre_node in pre_nodes:
            if node_label.endswith('end'):
                attrib_data[(pre_node, node)] = {"label": 'ep'}                    
            else:   
                attrib_data[(pre_node, node)] = {"label": node_label}
    return attrib_data

In [ ]:
def merge_graph(graph_data, label_data, func_name):
    '''
    Once the inst-func data is obtained this function is called which then merges
    are the nops in the data finally giving us structred representation of graph.
    '''
    f_name, f_addr = func_name.split('-')
    temp_label_data = {}
    in_nodes, out_nodes = get_degree_nodes(graph_data)
    global global_address_counter
    done = True
    while done:
        done = False
        nodes_data = find_mergable_nodes(graph_data, label_data)
        if nodes_data:
            done = True
            graph_data = merge_nodes([nodes_data[0]], graph_data)
    attr_data = to_edge_based(graph_data, label_data, func_name)
    nx.set_edge_attributes(graph_data, attr_data)
    n_nodes = len(graph_data.nodes())
    if not n_nodes:
        graph_data.add_node(global_address_counter)
        temp_label_data[global_address_counter]=f'{f_name}-start'
        global_address_counter += 1
        temp_label_data[global_address_counter]=f'{f_name}-end'
        global_address_counter += 1
        graph_data.add_edge(global_address_counter-2, global_address_counter-1, label='ep')
    else:
        in_nodes, out_nodes = get_degree_nodes(graph_data)
        if not len(in_nodes):
            in_nodes = [sorted(graph_data.nodes())[-1]]
        graph_data.add_node(global_address_counter)
        for in_node in in_nodes:
            graph_data.add_edge(global_address_counter, in_node, label=label_data[in_node])
        temp_label_data[global_address_counter]=f'{f_name}-start'
        global_address_counter += 1
        if len(out_nodes) > 1:
            for out_node in out_nodes:
                graph_data.add_edge(out_node, global_address_counter, label='ep')
            temp_label_data[global_address_counter]=f'{f_name}-end'
            global_address_counter += 1
    return graph_data, temp_label_data

In [ ]:
def replace_edge_label(graph_data, current_edge_value, label_data, ignore_list):
    """
    Add '_syscall' to valid syscall edges which helps in differentiating between system call and library call.
    """
    new_edge_data = {}
    new_edge_value = f"{current_edge_value.split('-')[0]}_syscall"
    for e1, e2 in graph_data.edges():
        label_value = graph_data[e1][e2]['label']
        if label_value == current_edge_value:
            new_edge_data[(e1, e2)] = {'label': new_edge_value}
        elif (label_value == 'ep') or label_value.endswith('-end') or label_value.startswith('jmp'):
            new_edge_data[(e1, e2)] = {'label': 'ep'}
    nx.set_edge_attributes(graph_data, new_edge_data)
    return graph_data

In [ ]:
def get_valid_func_nodes(functions, valid_sys_calls):
    """
    Keeps only those function nodes which calls a system call at the end.
    """
    cyclic_functions = set()
    valid_func_nodes = []
    for func_node in functions:
        if (func_node.is_syscall and (func_node.name in valid_sys_calls)):
            valid_func_nodes.append(func_node)
        elif len(func_node.get_call_sites()):
            valid_func_nodes.append(func_node)
    # Eliminating the functions which need help of dataflow analysis to know which func they are 
    # calling Ex :- __tfind.
    done = False
    while True:
        valid_func_nodes_cpy = valid_func_nodes.copy()
        if done:
            break
        done = True
        for node in valid_func_nodes_cpy:
            if node.is_syscall:
                continue
            edge_data = list(node.transition_graph.edges() - node.graph.edges())
            valid_func_length = len(edge_data)
            curr_invalid_length = 0            
            for e1, e2 in edge_data:
                if e2 not in valid_func_nodes_cpy:
                    curr_invalid_length += 1                    
            if curr_invalid_length == valid_func_length:
                valid_func_nodes.remove(node)
                done = False
    valid_func_node_names = set()
    to_remove = []
    for index, func_node_data in enumerate(valid_func_nodes):
        if func_node_data.name in cyclic_functions:
            to_remove.append(index)
            continue
        valid_func_node_names.add(func_node_data.name)
    for index in sorted(to_remove, reverse=True):
        del valid_func_nodes[index]
    return valid_func_nodes, valid_func_node_names

In [ ]:
def get_adjacent_node(node_value, _type, graph):
    """
    Returns node which form a edge based on the type provided by user.
    """
    if _type == 'pre':
        return [n2 for n1, n2 in graph.edges() if n1 == node_value]
    else:
        return [n1 for n1, n2 in graph.edges() if n2 == node_value]

In [ ]:
def remove_epsilon_nodes(G):
    """
    Different conditions that are used to remove epsilon edges from the graph to as extent as possible.
    """
    changed = False
    _break = False
    while True:
        data = edges_with_epsilon_remove(G)
        to_remove_node = None
        current_type = None
        for n1, n2, _type in data:
            if _type is None:
                _break = True
                break
            elif _type == 'self':
                G.remove_edge(n1, n2)
            elif _type == 'pre':
                current_type = 'pre'
                to_remove_node = n1
                pre_nodes = list(G.predecessors(n1))
                G.remove_edge(n1, n2)
                for node_x in pre_nodes:
                    G.add_edge(node_x, n2, label=G[node_x][n1]['label'])
            else:
                current_type = 'succ'
                to_remove_node = n2
                succ_nodes = list(G.successors(n2))
                G.remove_edge(n1, n2)
                for node_x in succ_nodes:
                    G.add_edge(n1, node_x, label=G[n2][node_x]['label'])
        if to_remove_node is not None:
            changed = True
            nodes = get_adjacent_node(to_remove_node, current_type, G)
            if current_type == 'succ':
                for node_v in nodes:
                    G.remove_edge(node_v, to_remove_node)
            else:
                for node_v in nodes:
                    G.remove_edge(to_remove_node, node_v)
            G.remove_node(to_remove_node)
        if _break:
            break
    return G, changed

In [ ]:
def edges_with_epsilon_remove(G):
    """
    Helper function which calculates the edges that need to be removed and returns the required data.
    """
    _type = None
    in_nodes, out_nodes = get_degree_nodes(G)
    in_node, out_node = None, None
    if len(in_nodes):
        in_node = in_nodes[0]
    if len(out_nodes):
        out_node = out_nodes[0]
    edges_data = list(G.edges())
    if len(edges_data) == 1:
        return [(None, None, _type)]
    for n1, n2 in edges_data:
        try:
            label_value = G[n1][n2]['label']
        except Exception as e:
            print(n1, n2)
            import pdb;pdb.set_trace()
        if label_value == 'ep':
            if n1 == n2:
                return [(n1, n2 , 'self')]
            else:
                n1_pre_nodes = list(G.predecessors(n1))
                n1_succ_nodes = list(G.successors(n1))
                n2_pre_nodes = list(G.predecessors(n2))
                n2_succ_nodes = list(G.successors(n2))
                n1_pre_len, n2_pre_len, n1_succ_len, n2_succ_len = len(n1_pre_nodes), len(n2_pre_nodes), len(n1_succ_nodes), len(n2_succ_nodes)
                if len(set(n1_pre_nodes) & set(n2_succ_nodes)):
                    continue
                if ((not n1_pre_len) and (n1_succ_len == 1)) or ((n1_pre_len == 1) and (n1_succ_len == 1) and (n2_succ_len == 1)):
                    return [(n1, n2, 'pre')]
                if ((not n2_succ_len) and (n2_pre_len == 1)) or ((n2_pre_len == 1) and (n2_succ_len == 1) and (n1_succ_len == 1)):
                    return [(n1, n2, 'succ')]
                if n2_succ_len < n1_pre_len:
                    if n2_pre_len == 1:
                        return [(n1, n2, 'succ')]
                    else:
                        pass
                if n1_pre_len < n2_succ_len:
                    if n1_succ_len == 1:
                        return [(n1, n2, 'pre')]
                    else:
                        pass
                if (n1_succ_len == 1) and (not len((set(n2_pre_nodes) - set([n1])) & set(n1_pre_nodes))):
                    return [(n1, n2, 'pre')]
                if (n2_pre_len == 1) and (not len((set(n1_succ_nodes) - set([n2])) & set(n2_succ_nodes))):
                    return [(n1, n2, 'succ')]
                if n1_pre_len == n2_succ_len:
                    continue
    return [(None, None, _type)]

In [ ]:
def remove_clique_epsilon_edges(graph_data, to_remove_edges, nodes_data, in_node, out_node, type_):
    """
    Removes epsilon edges that form a subgraph altogether.
    """
    other_nodes = nodes_data - set([in_node, out_node])
    for n1, n2 in to_remove_edges:
        graph_data.remove_edge(n1, n2)
    for node_data in other_nodes:
        other_pre_nodes = set(graph_data.predecessors(node_data)) - nodes_data
        for node_x in other_pre_nodes:
            graph_data.add_edge(node_x, out_node, label=graph_data[node_x][node_data]['label'])
        graph_data.remove_node(node_data)
    graph_data.add_edge(in_node, out_node, label="ep")
    return graph_data

In [ ]:
def remove_clique_epsilons(graph_data, dir_):
    """
    Helper function which calculates which subgraphs form epislon stack and sends them to remove them accordingly.
    """
    edges_data = [(n1, n2) for n1, n2 in graph_data.edges() if graph_data[n1][n2]['label'] == 'ep']
    edge_graph_data = graph_data.edge_subgraph(edges_data)
    T = edge_graph_data.to_undirected()
    success = False
    for dc_graph_nodes in nx.connected_components(T):
        dc_graph = edge_graph_data.subgraph(dc_graph_nodes)
        in_nodes, out_nodes = get_degree_nodes(dc_graph)
        non_epsilon = None
        if (len(in_nodes) == 1) and (len(out_nodes) == 1) and (len(dc_graph.edges()) > 1):
            in_node, out_node = in_nodes[0], out_nodes[0]
            failed = False
            other_nodes = dc_graph_nodes - set([in_node, out_node])
            for other_node in other_nodes:
                node_failure = False
                for node_x in graph_data.successors(other_node):
                    if graph_data[other_node][node_x]['label'] != 'ep':
                        node_failure = True
                        failed = True
                        break
                if node_failure:
                    break
                for node_x in graph_data.predecessors(other_node):
                    if graph_data[node_x][other_node]['label'] != 'ep':
                        node_failure = True
                        failed = True
                        break
                if node_failure:
                    break
            if failed:
                continue
            for data in nx.all_simple_edge_paths(graph_data, in_nodes[0], out_nodes[0]):
                for new_n1, new_n2 in data:
                    if graph_data[new_n1][new_n2]['label'] != 'ep':
                        non_epsilon = True
                        break
                if non_epsilon:
                    break
        else:
            non_epsilon = True
        if non_epsilon:
            continue
        success = True
        return remove_clique_epsilon_edges(graph_data, list(dc_graph.edges()), dc_graph_nodes, in_nodes[0], out_nodes[0], 'undir'), False
    if (not success) and dir_:
        imp_data = [data for data in list(nx.strongly_connected_components(edge_graph_data)) if len(data) > 1 ]
        for imp_nodes_data in imp_data:
            directed_graph = graph_data.subgraph(imp_nodes_data)
            in_node, out_node = None, None
            for node_data in imp_nodes_data:
                succ_data = set(graph_data.successors(node_data))
                pre_data = set(graph_data.predecessors(node_data))
                if len(succ_data - imp_nodes_data):
                    out_node = node_data
                if (out_node != node_data) and len(pre_data - imp_nodes_data):
                    in_node = node_data
            if (in_node is not None) and (out_node is not None):
                return remove_clique_epsilon_edges(graph_data, list(directed_graph.edges()), imp_nodes_data, in_node, out_node, 'dir'), True
    return graph_data, False

In [ ]:
def is_epsilon_removal_possible(graph_data, dir_=True):
    """
    Functions which calls the required epsilon removal functions..
    """
    current_count = 0
    changed = False
    status = True
    while status:
        graph_data, status = remove_clique_epsilons(graph_data, dir_=dir_)
        if (not status) and current_count:
            changed = True
        current_count += 1
    return graph_data, changed

### Program Start

In [ ]:
exe_file_name = os.environ.get('EXE_FILE', None)
if exe_file_name is None:
    print('Missing executable file name..')

In [ ]:
final_data = {}
global_label_data = {}
resolution_func_data = {}
resolved_data = []
counter2 = 0
global_address_counter = 0
valid_sys_calls = '''read,write,open,close,stat,fstat,lstat,poll,lseek,mmap,mprotect,munmap,brk,rt_sigaction,rt_sigprocmask,rt_sigreturn,ioctl,pread64,pwrite64,readv,writev,access,pipe,select,sched_yield,mremap,msync,mincore,madvise,shmget,shmat,shmctl,dup,dup2,pause,nanosleep,getitimer,alarm,setitimer,getpid,sendfile,socket,connect,accept,sendto,recvfrom,sendmsg,recvmsg,shutdown,bind,listen,getsockname,getpeername,socketpair,setsockopt,getsockopt,clone,fork,vfork,execve,exit,wait4,kill,uname,semget,semop,semctl,shmdt,msgget,msgsnd,msgrcv,msgctl,fcntl,flock,fsync,fdatasync,truncate,ftruncate,getdents,getcwd,chdir,fchdir,rename,mkdir,rmdir,creat,link,unlink,symlink,readlink,chmod,fchmod,chown,fchown,lchown,umask,gettimeofday,getrlimit,getrusage,sysinfo,times,ptrace,getuid,syslog,getgid,setuid,setgid,geteuid,getegid,setpgid,getppid,getpgrp,setsid,setreuid,setregid,getgroups,setgroups,setresuid,getresuid,setresgid,getresgid,getpgid,setfsuid,setfsgid,getsid,capget,capset,rt_sigpending,rt_sigtimedwait,rt_sigqueueinfo,rt_sigsuspend,sigaltstack,utime,mknod,uselib,personality,ustat,statfs,fstatfs,sysfs,getpriority,setpriority,sched_setparam,sched_getparam,sched_setscheduler,sched_getscheduler,sched_get_priority_max,sched_get_priority_min,sched_rr_get_interval,mlock,munlock,mlockall,munlockall,vhangup,modify_ldt,pivot_root,_sysctl,prctl,arch_prctl,adjtimex,setrlimit,chroot,sync,acct,settimeofday,mount,umount2,swapon,swapoff,reboot,sethostname,setdomainname,iopl,ioperm,create_module,init_module,delete_module,get_kernel_syms,query_module,quotactl,nfsservctl,getpmsg,putpmsg,afs_syscall,tuxcall,security,gettid,readahead,setxattr,lsetxattr,fsetxattr,getxattr,lgetxattr,fgetxattr,listxattr,llistxattr,flistxattr,removexattr,lremovexattr,fremovexattr,tkill,time,futex,sched_setaffinity,sched_getaffinity,set_thread_area,io_setup,io_destroy,io_getevents,io_submit,io_cancel,get_thread_area,lookup_dcookie,epoll_create,epoll_ctl_old,epoll_wait_old,remap_file_pages,getdents64,set_tid_address,restart_syscall,semtimedop,fadvise64,timer_create,timer_settime,timer_gettime,timer_getoverrun,timer_delete,clock_settime,clock_gettime,clock_getres,clock_nanosleep,exit_group,epoll_wait,epoll_ctl,tgkill,utimes,vserver,mbind,set_mempolicy,get_mempolicy,mq_open,mq_unlink,mq_timedsend,mq_timedreceive,mq_notify,mq_getsetattr,kexec_load,waitid,add_key,request_key,keyctl,ioprio_set,ioprio_get,inotify_init,inotify_add_watch,inotify_rm_watch,migrate_pages,openat,mkdirat,mknodat,fchownat,futimesat,newfstatat,unlinkat,renameat,linkat,symlinkat,readlinkat,fchmodat,faccessat,pselect6,ppoll,unshare,set_robust_list,get_robust_list,splice,tee,sync_file_range,vmsplice,move_pages,utimensat,epoll_pwait,signalfd,timerfd_create,eventfd,fallocate,timerfd_settime,timerfd_gettime,accept4,signalfd4,eventfd2,epoll_create1,dup3,pipe2,inotify_init1,preadv,pwritev,rt_tgsigqueueinfo,perf_event_open,recvmmsg,fanotify_init,fanotify_mark,prlimit64,name_to_handle_at,open_by_handle_at,clock_adjtime,syncfs,sendmmsg,setns,getcpu,process_vm_readv,process_vm_writev,kcmp,finit_module,sched_setattr,sched_getattr,renameat2,seccomp,getrandom,memfd_create,kexec_file_load,bpf,execveat,userfaultfd,membarrier,mlock2,copy_file_range,preadv2,pwritev2,pkey_mprotect,pkey_alloc,pkey_free,statx'''.split(',')
valid_sys_calls = set(valid_sys_calls)
different_jumps = ['call', 'jmp', 'jo', 'jno', 'js', 'jns', 'je',
                   'jz', 'jne', 'jnz', 'jb', 'jnae', 'jc', 'jnb',
                   'jae', 'jnc', 'jbe', 'jna', 'ja', 'jnbe', 'jle',
                   'jnge', 'jge', 'jnl', 'jl', 'jng', 'jg', 'jnle'
                  'jp', 'jpe', 'jnp', 'jpo', 'jcxz', 'jecxz', 'jrcxz']
different_jumps.sort(key = lambda x: (-len(x), x))
jumps_string = '|'.join(different_jumps)

In [ ]:
print('Starting angr binary analysis')
start_time = time()
proj = angr.Project(exe_file_name, auto_load_libs=False)
cfg = proj.analyses.CFGFast(normalize=True)
print(f'Finished getting data from angr.. in {time()-start_time}')

In [ ]:
functions = cfg.functions.values()
graph = cfg.graph.copy()

In [ ]:
valid_func_nodes, valid_func_node_names = get_valid_func_nodes(functions, valid_sys_calls)
func_addr_mapping = {}
for func_node in valid_func_nodes:
    f_name, f_addr = func_node.name, func_node.addr
    func_addr_mapping[f_name] = func_addr_mapping.get(f_name, []) + [f_addr]

In [ ]:
sys_calls = []
sys_calls_names = set()
sys_call_nodes = []
syscall_addr_mapping = {}
for node in graph.nodes:
    if node.is_syscall:
        sys_calls_names.add(f'{node.name}_syscall')
        sys_calls.append(f'{node}_syscall')
        sys_call_nodes.append(node)
        syscall_addr_mapping[node.name] = node.addr
assert len(sys_calls_names) == len(sys_calls)

In [ ]:
'''
Starting from the main function we are trying to resolve all the functions that are dependent on main
This process is repeated till we no longer see a function that is resolved.
Basically for every function we will be getting its high-level functions graph structure
'''
start_func = 'main'
to_resolve = [f'{start_func}-{func_addr_mapping[start_func][0]}']
visited = set()
while True:
    if not len(to_resolve):
        break
    current_func = to_resolve.pop(0)
    if current_func in visited:
        continue
    visited.add(current_func)
    f_name, f_addr = current_func.split('-')
    func_func_node = get_node_with_func_name(cfg, f_name, f_addr)
    if func_func_node.is_syscall:
        resolved_data.append(f'{current_func}')
        continue
    current_label_data, node_to_resolve = get_graph_label_data(cfg, current_func, valid_func_node_names, func_addr_mapping, syscall_addr_mapping)
    if len(node_to_resolve) == 0:
        resolved_data.append(f'{current_func}')
    else:
        for func_name in node_to_resolve:
            resolution_func_data[func_name] = resolution_func_data.get(func_name, set()) | {current_func}
        to_resolve += list(node_to_resolve)
    final_func_graph, temp_l_d = merge_graph(func_func_node.subgraph(current_label_data.keys()), current_label_data, current_func)
    current_label_data = {key: value for key, value in current_label_data.items() if key in final_func_graph.nodes()}
    global_label_data = {**global_label_data, **current_label_data, **temp_l_d}
    final_data[f'{current_func}'] = {'graph': final_func_graph, 'to_resolve': node_to_resolve}

In [ ]:
'''
Removing epsilon edges function based
'''
changed = True
while changed:
    changed = False
    status1 = False
    status2 = False
    for key, value in final_data.items():
        final_data[key]['graph'], c_status1 = remove_epsilon_nodes(final_data[key]['graph'])
        final_data[key]['graph'], c_status2 = is_epsilon_removal_possible(final_data[key]['graph'])
        if c_status1:
            status1 = True
        if c_status2:
            status2 = True
    changed = changed or status1 or status2

In [ ]:
count_data = Counter([f_name.split('-')[0] for f_name in final_data.keys()])
data = {name for name, value in count_data.items() if value > 1}
ignore_list = {name for name in final_data.keys() if name.split('-')[0] in data}

In [ ]:
# final_data :- 
#     Dict which contains key (func_data), value is a dict
#     value :- key1 (graph),value contains the di-graph object
#              key2 (to_resolve), value contains all the funcs by func_data
#     Ex :- {'main-4200660': {'graph': <networkx.classes.digraph.DiGraph at 0x7fffc01c4730>,
#                             'to_resolve': {'_IO_printf-4234848', 'foo-4200555'}}

# global_label_data :- 
#     key(add), value (correpsonding node value)
#     Ex :- {0x4018f2: 'foo-4200555'}
    
# resolution_func_data :- 
#     Contains data of functions that can be resolved if the key is already resolved.
#     Ex :- {'_IO_printf-4234848': {'bar-4200405', 'main-4200660'}} If _IO_printf is already we 
#     can resolve the main and bar as they contain call to _IO_printf

# resolved_data :- 
#     List of all functions names that are resolved (includes sys_calls)
#     Ex :-['futex-7340619', 'close-7340420']

In [ ]:
info_data = {}
number_of_graphs = {}
for key, value in final_data.items():
    G = value['graph']
    s_value = nx.number_connected_components(G.to_undirected())
    in_nodes, out_nodes = get_degree_nodes(G)
    if s_value > 1:
        number_of_graphs[key] = s_value
    if (len(in_nodes) > 1) or (len(out_nodes) > 1) or (not (len(in_nodes) or len(out_nodes))):
        info_data[key] = {'in_nodes': len(in_nodes), 'out_nodes': len(out_nodes)}

In [ ]:
'''
Starting with the resolved functions which no longer have any dependencies we loop through and
try to resolve other functions which can be resolved through the already resolved functions
'''
non_syscall_resolved = []
while (resolved_data != []):
    resolved_func = resolved_data.pop(0)
    resolved_func_data = final_data.get(resolved_func, None)
    if resolved_func_data is None:
        # system call edge replacemenet
        funcs_data = resolution_func_data[resolved_func]
        for func_value in funcs_data:
            graph_data = copy.deepcopy(final_data[func_value]['graph'])
            updated_graph = replace_edge_label(graph_data, resolved_func, global_label_data, ignore_list)
            final_data[func_value]['graph'] = updated_graph
            final_data[func_value]['to_resolve'].remove(resolved_func)
        del resolution_func_data[resolved_func]
    else:
        non_syscall_resolved.append(resolved_func)

In [ ]:
# Fixing Self loops so they can be used while traversing properly.
for key, value in final_data.items():
    graph_data = final_data[key]['graph']
    node_values = list(nx.nodes_with_selfloops(graph_data))
    fix_number = 0
    for node in node_values:
        label_value = graph_data[node][node]["label"]
        pre_data = (set(graph_data.predecessors(node)) - set([node]))
        succ_data = (set(graph_data.successors(node)) - set([node]))
        node1_name, func_name = f'{key}_ep{fix_number}', f'{key}_func{fix_number}',
        fix_number += 1
        graph_data.add_nodes_from([node1_name, func_name])
        graph_data.add_edge(node1_name, func_name, label=label_value)
        graph_data.add_edge(func_name, node1_name, label='ep')
        if pre_data:
            for pre_node in pre_data:
                pre_label_value = graph_data[pre_node][node]['label']
                graph_data.add_edge(pre_node, node1_name, label=pre_label_value)
                graph_data.remove_edge(pre_node, node)
        else:
            graph_data.add_node('start')
            graph_data.add_edge('start', node1_name, label='ep')
        if succ_data:
            for succ_node in succ_data:
                succ_label_value = graph_data[node][succ_node]['label']
                graph_data.add_edge(node1_name, succ_node,label=succ_label_value)
                graph_data.remove_edge(node, succ_node)  
        else:
            graph_data.add_node('end')
            graph_data.add_edge(func_name, 'end', label='ep')
        graph_data.remove_node(node)
        final_data[key]['graph'] = graph_data

In [ ]:
'''
Saving the graph_data
'''
final_graph_data = {}
for key, value in final_data.items():
    final_graph_data[key] = value['graph']
store_as_list_of_dicts(exe_file_name, final_graph_data)